In [1]:
import pandas as pd

In [2]:
dataset=pd.read_csv('filtered_ShubmanGill.csv')

In [3]:
dataset.drop(columns=['Unnamed: 0'],inplace=True)

In [4]:
dataset

,inns_number,bat,team,bowl,opposition,ball,ball_id,outs,dismissal,over,...,control,inns,1s,2s,3s,4s,5s,6s,bowler_dismissed?,dots
0,1.0,Shubman Gill,Kolkata Knight Riders,Billy Stanlake,Sunrisers Hyderabad,5.0,13.05,0,NaN,14.0,...,1.0,1,0,0,0,0,0,0,0,1
1,1.0,Shubman Gill,Kolkata Knight Riders,Rashid Khan,Sunrisers Hyderabad,4.0,15.04,0,NaN,16.0,...,1.0,1,0,0,0,0,0,0,0,1
2,1.0,Shubman Gill,Kolkata Knight Riders,Rashid Khan,Sunrisers Hyderabad,2.0,15.02,0,NaN,16.0,...,1.0,1,1,0,0,0,0,0,0,0
3,1.0,Shubman Gill,Kolkata Knight Riders,Siddarth Kaul,Sunrisers Hyderabad,5.0,14.05,0,NaN,15.0,...,1.0,1,1,0,0,0,0,0,0,0
4,1.0,Shubman Gill,Kolkata Knight Riders,Rashid Khan,Sunrisers Hyderabad,5.0,15.05,0,NaN,16.0,...,1.0,1,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2429,1.0,Shubman Gill,Gujarat Titans,Shardul Thakur,Chennai Super Kings,2.0,15.02,0,NaN,16.0,...,0.0,1,0,0,0,0,0,0,0,1
2430,1.0,Shubman Gill,Gujarat Titans,Shardul Thakur,Chennai Super Kings,3.0,15.03,0,NaN,16.0,...,1.0,1,0,1,0,0,0,0,0,0
2431,1.0,Shubman Gill,Gujarat Titans,Shardul Thakur,Chennai Super Kings,4.0,15.04,0,NaN,16.0,...,1.0,1,1,0,0,0,0,0,0,0
2432,1.0,Shubman Gill,Gujarat Titans,Tushar Deshpande,Chennai Super Kings,1.0,14.01,0,NaN,15.0,...,1.0,1,1,0,0,0,0,0,0,0


In [5]:
dataset.sort_values(by=['date','ball_id'],inplace=True)

In [6]:
dataset

,inns_number,bat,team,bowl,opposition,ball,ball_id,outs,dismissal,over,...,control,inns,1s,2s,3s,4s,5s,6s,bowler_dismissed?,dots
0,1.0,Shubman Gill,Kolkata Knight Riders,Billy Stanlake,Sunrisers Hyderabad,5.0,13.05,0,NaN,14.0,...,1.0,1,0,0,0,0,0,0,0,1
6,1.0,Shubman Gill,Kolkata Knight Riders,Billy Stanlake,Sunrisers Hyderabad,6.0,13.06,0,NaN,14.0,...,0.0,1,0,0,0,0,0,0,0,1
3,1.0,Shubman Gill,Kolkata Knight Riders,Siddarth Kaul,Sunrisers Hyderabad,5.0,14.05,0,NaN,15.0,...,1.0,1,1,0,0,0,0,0,0,0
2,1.0,Shubman Gill,Kolkata Knight Riders,Rashid Khan,Sunrisers Hyderabad,2.0,15.02,0,NaN,16.0,...,1.0,1,1,0,0,0,0,0,0,0
1,1.0,Shubman Gill,Kolkata Knight Riders,Rashid Khan,Sunrisers Hyderabad,4.0,15.04,0,NaN,16.0,...,1.0,1,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2382,1.0,Shubman Gill,Gujarat Titans,Simarjeet Singh,Chennai Super Kings,3.0,16.03,0,NaN,17.0,...,1.0,1,1,0,0,0,0,0,0,0
2383,1.0,Shubman Gill,Gujarat Titans,Simarjeet Singh,Chennai Super Kings,5.0,16.05,0,NaN,17.0,...,1.0,1,1,0,0,0,0,0,0,0
2424,1.0,Shubman Gill,Gujarat Titans,Tushar Deshpande,Chennai Super Kings,1.0,17.01,0,NaN,18.0,...,1.0,1,1,0,0,0,0,0,0,0
2425,1.0,Shubman Gill,Gujarat Titans,Tushar Deshpande,Chennai Super Kings,4.0,17.04,0,NaN,18.0,...,1.0,1,1,0,0,0,0,0,0,0


In [7]:
numeric_cols=dataset.groupby(['date','opposition','ground'])['runs','balls','outs','dots','1s','2s','3s','4s','5s','6s','control'].cumsum().reset_index()
nonnumeric_cols=dataset[['date','team','inns_number','opposition','ground','dismissal']]

C:\Users\Vijay\AppData\Local\Temp\ipykernel_12664\655963766.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  numeric_cols=dataset.groupby(['date','opposition','ground'])['runs','balls','outs','dots','1s','2s','3s','4s','5s','6s','control'].cumsum().reset_index()


In [8]:
cumulativedataset=pd.concat([nonnumeric_cols,numeric_cols],axis=1)

In [9]:
cumulativedataset

,date,team,inns_number,opposition,ground,dismissal,index,runs,balls,outs,dots,1s,2s,3s,4s,5s,6s,control
0,2018-04-14,Kolkata Knight Riders,1.0,Sunrisers Hyderabad,"Eden Gardens, Kolkata",NaN,0,0.0,1.0,0,1,0,0,0,0,0,0,1.0
6,2018-04-14,Kolkata Knight Riders,1.0,Sunrisers Hyderabad,"Eden Gardens, Kolkata",NaN,7,3.0,7.0,0,4,3,0,0,0,0,0,6.0
3,2018-04-14,Kolkata Knight Riders,1.0,Sunrisers Hyderabad,"Eden Gardens, Kolkata",NaN,2,2.0,4.0,0,2,2,0,0,0,0,0,3.0
2,2018-04-14,Kolkata Knight Riders,1.0,Sunrisers Hyderabad,"Eden Gardens, Kolkata",NaN,3,1.0,3.0,0,2,1,0,0,0,0,0,2.0
1,2018-04-14,Kolkata Knight Riders,1.0,Sunrisers Hyderabad,"Eden Gardens, Kolkata",NaN,6,0.0,2.0,0,2,0,0,0,0,0,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2382,2024-05-10,Gujarat Titans,1.0,Chennai Super Kings,"Narendra Modi Stadium, Ahmedabad",NaN,2416,12.0,6.0,0,2,2,0,0,1,0,1,5.0
2383,2024-05-10,Gujarat Titans,1.0,Chennai Super Kings,"Narendra Modi Stadium, Ahmedabad",NaN,2417,16.0,7.0,0,2,2,0,0,2,0,1,5.0
2424,2024-05-10,Gujarat Titans,1.0,Chennai Super Kings,"Narendra Modi Stadium, Ahmedabad",NaN,2430,95.0,46.0,0,8,21,3,0,8,0,6,39.0
2425,2024-05-10,Gujarat Titans,1.0,Chennai Super Kings,"Narendra Modi Stadium, Ahmedabad",NaN,2431,96.0,47.0,0,8,22,3,0,8,0,6,40.0


In [10]:
cumulativedataset['SR']=round(cumulativedataset['runs']*100/cumulativedataset['balls'],2)
cumulativedataset['Control%']=round(cumulativedataset['control']*100/cumulativedataset['balls'],2)
cumulativedataset.drop(['control'],axis=1,inplace=True)

In [11]:
cumulativedataset.drop(columns=['index'],inplace=True)

In [12]:
cumulativedataset

,date,team,inns_number,opposition,ground,dismissal,runs,balls,outs,dots,1s,2s,3s,4s,5s,6s,SR,Control%
0,2018-04-14,Kolkata Knight Riders,1.0,Sunrisers Hyderabad,"Eden Gardens, Kolkata",NaN,0.0,1.0,0,1,0,0,0,0,0,0,0.00,100.00
6,2018-04-14,Kolkata Knight Riders,1.0,Sunrisers Hyderabad,"Eden Gardens, Kolkata",NaN,3.0,7.0,0,4,3,0,0,0,0,0,42.86,85.71
3,2018-04-14,Kolkata Knight Riders,1.0,Sunrisers Hyderabad,"Eden Gardens, Kolkata",NaN,2.0,4.0,0,2,2,0,0,0,0,0,50.00,75.00
2,2018-04-14,Kolkata Knight Riders,1.0,Sunrisers Hyderabad,"Eden Gardens, Kolkata",NaN,1.0,3.0,0,2,1,0,0,0,0,0,33.33,66.67
1,2018-04-14,Kolkata Knight Riders,1.0,Sunrisers Hyderabad,"Eden Gardens, Kolkata",NaN,0.0,2.0,0,2,0,0,0,0,0,0,0.00,50.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2382,2024-05-10,Gujarat Titans,1.0,Chennai Super Kings,"Narendra Modi Stadium, Ahmedabad",NaN,12.0,6.0,0,2,2,0,0,1,0,1,200.00,83.33
2383,2024-05-10,Gujarat Titans,1.0,Chennai Super Kings,"Narendra Modi Stadium, Ahmedabad",NaN,16.0,7.0,0,2,2,0,0,2,0,1,228.57,71.43
2424,2024-05-10,Gujarat Titans,1.0,Chennai Super Kings,"Narendra Modi Stadium, Ahmedabad",NaN,95.0,46.0,0,8,21,3,0,8,0,6,206.52,84.78
2425,2024-05-10,Gujarat Titans,1.0,Chennai Super Kings,"Narendra Modi Stadium, Ahmedabad",NaN,96.0,47.0,0,8,22,3,0,8,0,6,204.26,85.11
